<a href="https://colab.research.google.com/github/agarwalsourabh55/huggingface-tapas/blob/sourabh_dev/huggingface_tapas_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install transformers torch-scatter

!pip install transformers
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+${CUDA}.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.9.0+.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=3567064 sha256=9d67f140dfa659898d5125c7283c689ed97b7462e1c23b64decf0e4f6dd52e76
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter


### Different Types of Declaring TAPAS models


1.   Default SQA Configutaion
2.   with WTQ Configuration
3.   With WikiSQL Configuration


In [ ]:
from transformers import TapasConfig, TapasForQuestionAnswering

# for example, the base sized model with default SQA configuration
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base")

# or, the base sized model with WTQ configuration
config = TapasConfig.from_pretrained("google/tapas-base-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)

# # or, the base sized model with WikiSQL configuration
# config = TapasConfig( "google/tapas-base-finetuned-wikisql-supervised",
#                         num_aggregation_labels=3,
#                         average_logits_per_cell=True)
# model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_weights', 'column_output_bias', 'output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_weights', 'aggregation_classifier.weight', 'output_bias', 'aggregation_classifier.bias', 'column_output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



### Defining the Tokenizer class


1.   Defining the data labels.
2.   Defining the querie.
3.   Defining the answers of that queries.
4.   Converting all into the DAtaframe if not.
5.   Tokenizer Takes Text Only. 
6.   Advised to use Dataloaders for training the TAPAS



In [ ]:
from transformers import TapasTokenizer
import pandas as pd

model_name = "google/tapas-base"
tokenizer = TapasTokenizer.from_pretrained(model_name)

data = {"Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], "Number of movies": ["87", "53", "69"]}
queries = [
    "What is the name of the first actor?",
    "How many movies has George Clooney played in?",
    "What is the total number of movies?",
]
answer_coordinates = [[(0, 0)], [(2, 1)], [(0, 1), (1, 1), (2, 1)]]
answer_text = [["Brad Pitt"], ["69"], ["209"]]
table = pd.DataFrame.from_dict(data)
inputs = tokenizer(
    table=table,
    queries=queries,
    answer_coordinates=answer_coordinates,
    answer_text=answer_text,
    padding="max_length",
    return_tensors="pt",
)
inputs

{'input_ids': tensor([[ 101, 2054, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2116,  ...,    0,    0,    0],
        [ 101, 2054, 2003,  ...,    0,    0,    0]]), 'labels': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'numeric_values': tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]]), 'numeric_values_scale': tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]), 'token_type_ids': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [ ]:
table

,Actors,Number of movies
0,Brad Pitt,87
1,Leonardo Di Caprio,53
2,George Clooney,69


## TAPAS for Question Answering pretrained

Using Different methodology to defint the predefine the logits

In [ ]:
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd

tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq")

data = {
    "Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"],
    "Age": ["56", "45", "59"],
    "Number of movies": ["87", "53", "69"],
}
table = pd.DataFrame.from_dict(data)
queries = ["How many movies has George Clooney played in?", "How old is Brad Pitt?"]

inputs = tokenizer(table=table, queries=queries, padding="max_length", return_tensors="pt")
outputs = model(**inputs)

logits = outputs.logits
logits_aggregation = outputs.logits_aggregation

### Taking the Inference of TAPAS Pretrained models using the cricketers data.
Conducted a small POC to check how accurately TAPAS model used the predefined modes and predict the modelling approach

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
data = pd.read_csv("/content/gdrive/MyDrive/graduate_rate.csv")
data = data.astype(str)

Mounted at /content/gdrive


In [ ]:
data

,Reporting_Level,System_ID,School_ID,System_Name,School_Name,Reporting_Label,Graduation_Rate
0,School,601,0103,Appling County,Appling County High School,ALL Students,93.18
1,School,601,0103,Appling County,Appling County High School,American Indian/Alaskan,Too Few Students
2,School,601,0103,Appling County,Appling County High School,Asian/Pacific Islander,Too Few Students
3,School,601,0103,Appling County,Appling County High School,Black,84.21
4,School,601,0103,Appling County,Appling County High School,Economically Disadvantaged,93.18
...,...,...,...,...,...,...,...
6678,State,ALL,ALL,All Systems,All Schools,English Learners,68.49
6679,State,ALL,ALL,All Systems,All Schools,Hispanic,80.68
6680,State,ALL,ALL,All Systems,All Schools,Multi-Racial,87.08
6681,State,ALL,ALL,All Systems,All Schools,Students With Disability,74.01


In [ ]:
data.dropna()

,Reporting_Level,System_ID,School_ID,System_Name,School_Name,Reporting_Label,Graduation_Rate
0,School,601,0103,Appling County,Appling County High School,ALL Students,93.18
1,School,601,0103,Appling County,Appling County High School,American Indian/Alaskan,Too Few Students
2,School,601,0103,Appling County,Appling County High School,Asian/Pacific Islander,Too Few Students
3,School,601,0103,Appling County,Appling County High School,Black,84.21
4,School,601,0103,Appling County,Appling County High School,Economically Disadvantaged,93.18
...,...,...,...,...,...,...,...
6678,State,ALL,ALL,All Systems,All Schools,English Learners,68.49
6679,State,ALL,ALL,All Systems,All Schools,Hispanic,80.68
6680,State,ALL,ALL,All Systems,All Schools,Multi-Racial,87.08
6681,State,ALL,ALL,All Systems,All Schools,Students With Disability,74.01


,Reporting_Level,System_ID,School_ID,System_Name,School_Name,Reporting_Label,Graduation_Rate
count,442,442,442,442,442,442,442
unique,1,188,200,188,433,1,360
top,School,667,0101,Gwinnett County,Lakeside High School,ALL Students,100.00
freq,442,26,14,26,2,442,18


In [ ]:
data

,Reporting_Level,System_ID,School_ID,System_Name,School_Name,Reporting_Label,Graduation_Rate
0,School,601,0103,Appling County,Appling County High School,ALL Students,93.18
1,School,601,0103,Appling County,Appling County High School,American Indian/Alaskan,Too Few Students
2,School,601,0103,Appling County,Appling County High School,Asian/Pacific Islander,Too Few Students
3,School,601,0103,Appling County,Appling County High School,Black,84.21
4,School,601,0103,Appling County,Appling County High School,Economically Disadvantaged,93.18
...,...,...,...,...,...,...,...
6678,State,ALL,ALL,All Systems,All Schools,English Learners,68.49
6679,State,ALL,ALL,All Systems,All Schools,Hispanic,80.68
6680,State,ALL,ALL,All Systems,All Schools,Multi-Racial,87.08
6681,State,ALL,ALL,All Systems,All Schools,Students With Disability,74.01


In [ ]:
# !pip install transformers

In [ ]:
# Defined the tapas model and the pipelinein which we are going to use it.
from transformers import pipeline
tqa = pipeline(task="table-question-answering", 
               model="google/tapas-base-finetuned-wtq")

ImportError: ignored

In [ ]:
!pip install torch-scatter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
data_soted_new

,System_Name,School_Name,Reporting_Label,Graduation_Rate
0,DeKalb County,Arabia Mountain High School - Academy of Engin...,ALL Students,99.67
1,Clayton County,Martha Ellen Stilwell School for the Performin...,ALL Students,99.28
2,Columbia County,Greenbrier High School,ALL Students,99.21
3,Atlanta Public Schools,Charles Drew Charter School JA/SA,ALL Students,99.10
4,Savannah-Chatham County,Savannah Arts Academy,ALL Students,99.10
...,...,...,...,...
437,Cobb County,Devereux Ackerman Academy,ALL Students,0.00
438,Savannah-Chatham County,UHS of Savannah Coastal Harbor Treatment Center,ALL Students,0.00
439,Taylor County,Georgia Center,ALL Students,0.00
440,Gwinnett County,Gwinnett Intervention Education Center (GIVE) ...,ALL Students,0.00


In [ ]:
data_new = data[(data['Reporting_Level']=='School') & (data['Reporting_Label']=='ALL Students')]
data_new1 = data_new[data_new['Graduation_Rate']!='Too Few Students']
data_sorted = data_new1.sort_values(by='Graduation_Rate',ascending = False)
data_soted_new = data_sorted[data_sorted['Graduation_Rate']!='No Data']
data_soted_new = data_soted_new.drop('Reporting_Level',axis=1)
data_soted_new.describe()
data_soted_new.drop(['System_ID','School_ID'],axis=1,inplace = True)
data_soted_new = data_soted_new.reset_index(drop='True')
all_new_data = pd.DataFrame(data_soted_new.to_dict('records'))

all_new_data.rename(column = {'System_Name':'system name',
                              'School_Name':'school name',
                              'Reporting_Label':'reporting',
                              'Graduation_Rate':'graduation'},inplace= True)

TypeError: ignored

In [ ]:
pos = range(1,443)
data_soted_new['Pos']=pos
data_soted_new.head()

,System_Name,School_Name,Reporting_Label,Graduation_Rate,Pos
0,DeKalb County,Arabia Mountain High School - Academy of Engin...,ALL Students,99.67,1
1,Clayton County,Martha Ellen Stilwell School for the Performin...,ALL Students,99.28,2
2,Columbia County,Greenbrier High School,ALL Students,99.21,3
3,Atlanta Public Schools,Charles Drew Charter School JA/SA,ALL Students,99.10,4
4,Savannah-Chatham County,Savannah Arts Academy,ALL Students,99.10,5


In [ ]:
data_soted_new['Pos'] = data_soted_new['Pos'].astype('str')

In [ ]:
qureies = ["what is the School_Name with highest Graduation_Rate"]
answer_list = tqa(table=data,
                  query=qureies,
                  )

In [ ]:
answer_list

{'answer': 'System',
 'coordinates': [(13, 0)],
 'cells': ['System'],
 'aggregator': 'NONE'}

In [ ]:
for answer in answer_list:
    print(answer["answer"])

1992-2007
Sachin Tendulkar
SUM > 18426, 14234, 13704, 13430, 12650, 11867, 11739, 11579, 11363, 10889
Sachin Tendulkar, Virat Kohli, Saurav Ganguly, Rahul Dravid


In [ ]:
answer

{'answer': 'Sachin Tendulkar, Virat Kohli, Saurav Ganguly, Rahul Dravid',
 'coordinates': [(0, 1), (5, 1), (8, 1), (9, 1)],
 'cells': ['Sachin Tendulkar',
  'Virat Kohli',
  'Saurav Ganguly',
  'Rahul Dravid'],
 'aggregator': 'NONE'}

### Tapas Modelling through the tokenizers

In [ ]:
!pip install torch-scatter transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.0 MB/s 
     |████████████████████████████████| 6.6 MB 43.2 MB/s 
     |████████████████████████████████| 120 kB 64.4 MB/s 
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=274491 sha256=0aefbb3925029d539b1ef67b06222f3d62e647aab71e0cfb3845efdfffbf0d7d
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter


In [ ]:
from transformers import AutoModelForTableQuestionAnswering, AutoTokenizer, pipeline
import pandas as pd

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
graduate_data = pd.read_csv("/content/gdrive/MyDrive/graduate_rate.csv")
graduate_data = graduate_data.astype(str)

Mounted at /content/gdrive


In [ ]:
graduate_data= graduate_data.astype(str)

In [ ]:
# Load model & tokenizer
model = 'google/tapas-large-finetuned-wtq'
model1= 'microsoft/tapex-large-finetuned-wtq'
model2 = 'google/tapas-large-finetuned-sqa'
tapas_model = AutoModelForTableQuestionAnswering.from_pretrained(model)
tapas_tokenizer = AutoTokenizer.from_pretrained(model)

# Initializing pipeline
tqa = pipeline('table-question-answering',
               model=tapas_model,
               tokenizer=tapas_tokenizer)

def questionanswering(query,data):
    response = tqa({'table': data,'query':query})
    answer = response['cells']
    print(answer)

In [ ]:
prediction = questionanswering('What is the School_Name with highest Graduataion_Rate',
                graduate_data[(graduate_data['Reporting_Level']=='School') & (graduate_data['Reporting_Label']=='ALL Students')].reset_index(drop=True).drop(['Reporting_Level','Reporting_Label'],axis=1).iloc[:310,:]
                )
#310 is the breaking point

[]


In [ ]:
graduate_data[(graduate_data['Reporting_Level']=='School') & (graduate_data['Reporting_Label']=='ALL Students')].reset_index(drop=True).drop(['Reporting_Level','Reporting_Label'],axis=1).iloc[:310,:]


,System_ID,School_ID,System_Name,School_Name,Graduation_Rate
0,601,0103,Appling County,Appling County High School,93.18
1,602,0103,Atkinson County,Atkinson County High School,94.35
2,603,0302,Bacon County,Bacon County High School,94.44
3,604,0105,Baker County,Baker County K12 School,Too Few Students
4,605,0189,Baldwin County,Baldwin High School,88.07
...,...,...,...,...,...
305,699,4050,Meriwether County,Manchester High School,76.79
306,700,0210,Miller County,Miller County High School,81.43
307,701,0282,Mitchell County,Mitchell County High School,87.21
308,702,0184,Monroe County,Mary Persons High School,92.11


Possible Reason for the Code Failure

1.   The vector size that the data going to create is not matching with the required model( most probably this is the reason, because when I am trying to feed less data it is ginv g me the response)
2.   The data type is not right




Next Steps:


*   will have to deifne the maximum size of tokens in tokenizer
*   have to tweak some of the parameters in model
*   Have to try different model of tapas(There are many different models that is included in the tapas library) to check for the accuracy



## Pipeline

In [2]:
!pip install torch-scatter transformers
!pip install huggingface-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.5 MB/s 
     |████████████████████████████████| 120 kB 48.9 MB/s 
     |████████████████████████████████| 6.6 MB 38.9 MB/s 
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=274491 sha256=24e4c60107f395bca11d204a5d0c2903e71dee17448fc18502da713616c1ca1e
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from transformers import AutoModelForTableQuestionAnswering, AutoTokenizer, pipeline, TapexTokenizer, BartForConditionalGeneration
import pandas as pd
from google.colab import drive

In [4]:
from transformers import AutoModelForTableQuestionAnswering, AutoTokenizer, pipeline, TapexTokenizer, BartForConditionalGeneration
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
graduate_data = pd.read_excel("/content/gdrive/MyDrive/gadoe_school_level.xlsx")
graduate_data = graduate_data.astype(str)

Mounted at /content/gdrive


In [5]:

from transformers import AutoModelForTableQuestionAnswering, AutoTokenizer, pipeline, TapexTokenizer, BartForConditionalGeneration
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
graduate_data = pd.read_excel("/content/gdrive/MyDrive/gadoe_school_level.xlsx")
graduate_data = graduate_data.astype(str)
dict1 = {
         'Tapas Large Wtq' : { "model_name":'google/tapas-large-finetuned-wtq'},
         }


for key,val in dict1.items():
  if 'Tapas' in key:
    tapas_model = AutoModelForTableQuestionAnswering.from_pretrained(val["model_name"])
    tapas_tokenizer = AutoTokenizer.from_pretrained(val["model_name"])
    val['tokenizer'] = tapas_tokenizer
    val['model'] = tapas_model
  else:
    tapex_tokenizer = TapexTokenizer.from_pretrained(val["model_name"]) #we can use medium as well
    tapex_model = BartForConditionalGeneration.from_pretrained(val["model_name"])
    val['tokenizer'] = tapex_tokenizer
    val['model'] = tapex_model

In [ ]:
# for key in list(dict1.keys())[-3:]:
#   val = dict1[key]
#   tapex_tokenizer = TapexTokenizer.from_pretrained(val["model_name"]) #we can use medium as well
#   tapex_model = BartForConditionalGeneration.from_pretrained(val["model_name"])
#   val['tokenizer'] = tapex_tokenizer
#   val['model'] = tapex_model
# def questionanswering(query,data):
#     tqa = pipeline('table-question-answering',
#                model=dict1['Tapas Large Wtq']['model'],
#                tokenizer=dict1['Tapas Large Wtq']['tokenizer'])
    
#     response = tqa({'table': data,'query':query})
#     print(response)



In [6]:


from transformers import AutoModelForTableQuestionAnswering, AutoTokenizer, pipeline, TapexTokenizer, BartForConditionalGeneration
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
graduate_data = pd.read_excel("/content/gdrive/MyDrive/gadoe_school_level.xlsx")
graduate_data = graduate_data.astype(str)
dict1 = {
         'Tapas Large Wtq' : { "model_name":'google/tapas-large-finetuned-wtq'},
         }


for key,val in dict1.items():
  if 'Tapas' in key:
    tapas_model = AutoModelForTableQuestionAnswering.from_pretrained(val["model_name"])
    tapas_tokenizer = AutoTokenizer.from_pretrained(val["model_name"])
    val['tokenizer'] = tapas_tokenizer
    val['model'] = tapas_model
  else:
    tapex_tokenizer = TapexTokenizer.from_pretrained(val["model_name"]) #we can use medium as well
    tapex_model = BartForConditionalGeneration.from_pretrained(val["model_name"])
    val['tokenizer'] = tapex_tokenizer
    val['model'] = tapex_model


## Data Cleaning Function
def data_clean(data):
  return data

cleaned_data = data_clean(graduate_data)


In [11]:
cleaned_data

,System_Name,School_Name,Graduation_Rate,Context
0,Carroll County,Mt. Zion High School,100.0,Mt. Zion High School in Carroll County has a g...
1,Savannah-Chatham County,Woodville-Tompkins Technical and Career High S...,100.0,Woodville-Tompkins Technical and Career High S...
2,Savannah-Chatham County,Savannah Early College High School,100.0,Savannah Early College High School in Savannah...
3,Clayton County,Elite Scholars Academy School,100.0,Elite Scholars Academy School in Clayton Count...
4,DeKalb County,Dekalb Early College Academy,100.0,Dekalb Early College Academy in DeKalb County ...
...,...,...,...,...
437,Savannah-Chatham County,UHS of Savannah Coastal Harbor Treatment Center,0.0,UHS of Savannah Coastal Harbor Treatment Cente...
438,Cobb County,Devereux Ackerman Academy,0.0,Devereux Ackerman Academy in Cobb County has a...
439,Gwinnett County,Gwinnett InterVention Education (GIVE) Center ...,0.0,Gwinnett InterVention Education (GIVE) Center ...
440,Gwinnett County,Gwinnett Intervention Education Center (GIVE) ...,0.0,Gwinnett Intervention Education Center (GIVE) ...


In [12]:
list_ofquestion = [
            {"question":"what is average graduate rate in Fulton Country?",
             "answer":"99.67"},
            {"question":"How many Schools in Fulton County?",
             "answer":"262"},
            {"question":"What is the lowest average graduate rate district name?",
             "answer":"All Students"}
            # {"question":"What is the count of System_Name for the Graduation_Rate above to 80",
            #  "answer":"380"},
            # {"question":"how many system_name are their for Clayton County",
            #  "answer":"12"},
            # {"question":"what is the maximum Graduation_Rate for School_name as Savannah Arts Academy",
            #  "answer":"99.10"}
            ]

def askQuestion(query,table,tokenizer,model):
    encoding = tokenizer(table=table, query=query, return_tensors="pt", max_length =1024, truncation=True)
    outputs = model.generate(**encoding)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

for key,val in dict1.items():
  ini_model = val['model']
  init_token = val['tokenizer']

  if 'Tapas' in key:
    tqa = pipeline('table-question-answering',
          model=ini_model,
          tokenizer=init_token)
    for question in list_ofquestion:
      try:
        response = tqa({'table': cleaned_data.iloc[:100,:],
                        'query':question['question']})
      except Exception as exp:
        print(exp)
      question[key] = response['answer']
  else:
    for question in list_ofquestion:
      response = askQuestion(query= question['question'],
                            table = cleaned_data.iloc[:100,:],
                            tokenizer = init_token,
                            model = ini_model )
      question[key] = response


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 1024 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [13]:
list_ofquestion

[{'question': 'what is average graduate rate in Fulton Country?',
  'answer': '99.67',
  'Tapas Large Wtq': 'AVERAGE > 100.0, 98.85, 98.2, 98.12, 97.76, 97.71, 97.44, 96.89, 96.89, 96.78, 96.75, 96.75, 96.75, 96.65, 96.62, 96.61, 96.52, 96.47, 96.43, 96.39, 96.39, 96.39, 96.37, 96.3, 96.28, 96.26, 96.17, 96.17, 96.15, 96.13, 96.07, 96.07, 96.02, 96.02',
  'Tapas Base': '',
  'Tapas base WTQ': 'AVERAGE > 98.85, 97.71',
  'Tapex Base WTQ': [' graduate rate of 100'],
  'Tapex Base wikisql': [' 100.0']},
 {'question': 'How many Schools in Fulton County?',
  'answer': '262',
  'Tapas Large Wtq': "COUNT > Corretta Scott King Womens' Leadership Academy, Johns Creek High School, Cambridge High School, Chattahoochee High School, Alpharetta High School, Northview High School, Cherokee Bluff High School",
  'Tapas Base': '97.14',
  'Tapas base WTQ': 'COUNT > Johns Creek High School, Cambridge High School, Chattahoochee High School, Alpharetta High School, Northview High School, Temple High School

In [38]:
ini_model = dict1['Tapas Large Wtq']['model']
init_token = dict1['Tapas Large Wtq']['tokenizer']
# if 'Tapas' in key:
tqa = pipeline('table-question-answering',
      model=ini_model,
      tokenizer=init_token)

question = "What is the average graduate_rate in Fulton County?"
response = tqa({'table': cleaned_data.iloc[:300,:],
                'query':question})

In [ ]:
!pip install gradio

In [8]:
import gradio as gr
def question_answer(question):
      ini_model = dict1['Tapas Large Wtq']['model']
      init_token = dict1['Tapas Large Wtq']['tokenizer']
      # if 'Tapas' in key:
      tqa = pipeline('table-question-answering',
            model=ini_model,
            tokenizer=init_token)
      response = tqa({'table': cleaned_data.iloc[:300,:],
                      'query':question})
      
      if response['aggregator']=="COUNT":
        answer = len(response['cells'])
      elif response['aggregator']=='AVERAGE':
        response['cells'] = [float(ele) for ele in response['cells']]
        answer = sum(response['cells']) / len(response['cells'])
      else:
        answer = response['answer']
      return answer

gr.Interface(fn=question_answer, inputs=["text"], outputs=[ "text"]).launch()


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://19940.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f81b3171d50>,
 'http://127.0.0.1:7860/',
 'https://19940.gradio.app')

### Demo Pipeline

In [9]:
!pip install torch-scatter transformers
!pip install huggingface-hub
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self

In [ ]:


from transformers import AutoModelForTableQuestionAnswering, AutoTokenizer, pipeline, TapexTokenizer, BartForConditionalGeneration
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
graduate_data = pd.read_excel("/content/gdrive/MyDrive/gadoe_school_level.xlsx")
graduate_data = graduate_data.astype(str)
dict1 = {
         'Tapas Large Wtq' : { "model_name":'google/tapas-large-finetuned-wtq'},
         }


for key,val in dict1.items():
  if 'Tapas' in key:
    tapas_model = AutoModelForTableQuestionAnswering.from_pretrained(val["model_name"])
    tapas_tokenizer = AutoTokenizer.from_pretrained(val["model_name"])
    val['tokenizer'] = tapas_tokenizer
    val['model'] = tapas_model
  else:
    tapex_tokenizer = TapexTokenizer.from_pretrained(val["model_name"]) #we can use medium as well
    tapex_model = BartForConditionalGeneration.from_pretrained(val["model_name"])
    val['tokenizer'] = tapex_tokenizer
    val['model'] = tapex_model




In [33]:
import gradio as gr
def question_answer(question):
      ini_model = dict1['Tapas Large Wtq']['model']
      init_token = dict1['Tapas Large Wtq']['tokenizer']
      # if 'Tapas' in key:
      tqa = pipeline('table-question-answering',
            model=ini_model,
            tokenizer=init_token)
      response = tqa({'table': graduate_data.iloc[:300,:],
                      'query':question})
      
      if response['aggregator']=="COUNT":
        answer = len(response['cells'])
      elif response['aggregator']=='AVERAGE':
        response['cells'] = [float(ele) for ele in response['cells']]
        answer = sum(response['cells']) / len(response['cells'])
      else:
        answer = response['answer']
      return answer

gr.Interface(fn=question_answer, inputs=["text"], outputs=[ "text"]).launch()


array(['Johns Creek High School', 'Cambridge High School',
       'Chattahoochee High School', 'Alpharetta High School',
       'Northview High School', 'Milton High School',
       'Westlake High School', 'Hapeville Charter Career Academy',
       'North Springs High School',
       'Riverwood International Charter School', 'Centennial High School',
       'Roswell High School'], dtype=object)

In [ ]:
results = pd.DataFrame(list_ofquestion)
results.to_csv('./gdrive/MyDrive/results.csv',index=False)